In [ ]:
# !python -m venv venv
# # !venv/scripts/activate
%pip install pandas numpy matplotlib seaborn plotly geopandas scipy scikit-learn statsmodels requests aiohttp

In [4]:
# Data Manipulation
import pandas as pd
pd.set_option('display.max_rows', None)

import numpy as np

# Data Visualization
import matplotlib.pyplot as plt



# Statistical Models
import statsmodels.api as sm

import requests
import os

import datetime as dt
import time

from pandas import read_pickle
import concurrent.futures
import psutil

from datetime import datetime
import re

In [ ]:
'''df = pd.read_json(os.path.join('aircraft_data_2024-02-01','aircraft_data_2024-02-01.json')
df = pd.json_normalize(df['aircraft'])
df'''

In [ ]:
# for each timeslice of 5 minutes, find the aircraft-ping with nac_p, nac_v above a certain threshold, 
# and gpsOkBefore, rr_lat, rr_lon if these values are not empty
# (each timeslice is 800kb)
#
# after filtering the timeslice, the filtered data should be much smaller say 10 kb
#
# define array of locations (squares) according to lat and lon limits
# 
# loop through all timeslice-aircraft-ping and find location and append
# 
# loop through all locations and run the data through the classification function to determine the color


In [11]:
# reduce pickle file size, select only useful columns 
def downsize_pickles(file_path):
    df = read_pickle(file_path)
    if 'dbFlags' in df:
        df = df[['time','hex','lat','lon','nic','type', 'flight', 'r', 't', 'category', 'version', 'nac_p', 'nac_v', 'roll' ]]
        df.to_pickle(file_path)

def str_to_datetime(str):
    pattern = r'(\d{4})(\d{2})(\d{2})\\(\d{2})(\d{2})(\d{2})Z'

    match = re.search(pattern, str)
    if match:
        year = int(match.group(1))
        month = int(match.group(2))
        day = int(match.group(3))
        hour = int(match.group(4))
        minute = int(match.group(5))
        second = int(match.group(6))

        dt = datetime(year, month, day, hour, minute, second)
        return dt
    else:
        print("No match found")

def add_time(file_path):
    df = read_pickle(file_path)
    # f'pickle/{start.strftime("%Y%m%d")}/{start.strftime("%H%M%S")}Z.pkl'
    file_time = str_to_datetime(file_path)
    print(file_time.strftime("%Y%m%dT%H%M%SZ"))
    df['time'] = file_time.strftime("%Y%m%dT%H%M%SZ")
    df = df[['time','hex','lat','lon','nic','type', 'flight', 'r', 't', 'category', 'version', 'nac_p', 'nac_v', 'roll' ]]
    df.to_pickle(file_path)

def process_files(directory, fn):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".pkl"):
                try:
                    file_path = os.path.join(root, file)
                    # Process the pickle file here
                    fn(file_path)
                    print(f"Processed {fn.__name__} {file_path}")
                except Exception as e:
                    # Handle other exceptions
                    print("An error occurred:", e)

# process_files("pickle", downsize_pickles)
process_files("pickle", add_time)

20240201T000000Z
Processed add_time pickle\20240201\000000Z.pkl
20240201T000005Z
Processed add_time pickle\20240201\000005Z.pkl
20240201T000010Z
Processed add_time pickle\20240201\000010Z.pkl
20240201T000015Z
Processed add_time pickle\20240201\000015Z.pkl
20240201T000020Z
Processed add_time pickle\20240201\000020Z.pkl
20240201T000025Z
Processed add_time pickle\20240201\000025Z.pkl
20240201T000030Z
Processed add_time pickle\20240201\000030Z.pkl
20240201T000035Z
Processed add_time pickle\20240201\000035Z.pkl
20240201T000040Z
Processed add_time pickle\20240201\000040Z.pkl
20240201T000045Z
Processed add_time pickle\20240201\000045Z.pkl
20240201T000050Z
Processed add_time pickle\20240201\000050Z.pkl
20240201T000055Z
Processed add_time pickle\20240201\000055Z.pkl
20240201T000100Z
Processed add_time pickle\20240201\000100Z.pkl
20240201T000105Z
Processed add_time pickle\20240201\000105Z.pkl
20240201T000110Z
Processed add_time pickle\20240201\000110Z.pkl
20240201T000115Z
Processed add_time pick

In [12]:
def time_elapsed(start_time):
    end_time = time.time()
    return end_time - start_time

time_data={
    "pickle": 0,
    "url": 0,
    "concat": 0,
}

def get_adsb_data(data):
    response = requests.get(data["url"])
    print(f"downloaded {data['url']}")
    json_data = response.json()
    
    utc_time = dt.datetime.fromtimestamp(json_data['now'])
    df = pd.json_normalize(json_data['aircraft'])

    # Create a new column for the primary key
    df['time'] = utc_time.strftime('%Y%m%dT%H%M%S')
    # reduce pickle file size, select only useful columns 
    df = df[['time','hex','lat','lon','nic','type', 'flight', 'r', 't', 'category', 'version', 'nac_p', 'nac_v', 'roll' ]]
    df = df.fillna("Nan values")
    df = df.map(lambda x: ', '.join(x) if isinstance(x, list) else x)

    #save to pickle
    if not os.path.exists(os.path.dirname(data["pickle"])):
        os.makedirs(os.path.dirname(data["pickle"]))
    df.to_pickle(data["pickle"])
    return df

def download_data(index, data, length_data_array, start_time_overall):
    start_time = time.time()

    try:
        # if pickle exists, do nothing
        if (os.path.isfile(data["pickle"])):
            print(f"pickle exists {data['pickle']}")
            return
        else:
            get_adsb_data(data)

            time_data["url"] += time_elapsed(start_time)

            remaining_time = (time.time() - start_time_overall)/index * (length_data_array - index)
            print(f"Remaining execution time: {remaining_time//60:.0f}m {remaining_time%60:.0f}s")

            return

    except Exception as e:
        # Handle other exceptions
        print("An error occurred:", e)

def read_data(index, data, length_data_array, start_time_overall):
    start_time = time.time()

    try:
        # if pickle exists, then load the pickle, otherwise, fetch from adsbexchange
        if (os.path.isfile(data["pickle"])):
            df = read_pickle(data["pickle"])
            print(f"loaded {data['pickle']}")
            time_data["pickle"] += time_elapsed(start_time)

            remaining_time = (time.time() - start_time_overall)/index * (length_data_array - index)
            print(f"Remaining execution time: {remaining_time//60:.0f}m {remaining_time%60:.0f}s")

            return df
        else:
            df = get_adsb_data(data)

            time_data["url"] += time_elapsed(start_time)
            
            remaining_time = (time.time() - start_time_overall)/index * (length_data_array - index)
            print(f"Remaining execution time: {remaining_time//60:.0f}m {remaining_time%60:.0f}s")
            return df

    except Exception as e:
        # Handle other exceptions
        print("An error occurred:", e)

def load_data(start_date_time,end_date_time):
    delta = dt.timedelta(seconds=5)
    start = start_date_time
    df1 = None

    df_array= []
    data_array = []
    while start <= end_date_time:
        data_array.append({
            "url": f'https://samples.adsbexchange.com/readsb-hist/{start.strftime("%Y/%m/%d")}/{start.strftime("%H%M%S")}Z.json.gz',
            "pickle": f'pickle/{start.strftime("%Y%m%d")}/{start.strftime("%H%M%S")}Z.pkl'
            })
        start += delta

    report1=""
    report2=""

    start_time_overall = time.time()

    with concurrent.futures.ThreadPoolExecutor() as executor:
        l = [len(data_array)] * len(data_array)
        list(executor.map(lambda i: download_data(i, data_array[i], l[i], start_time_overall), range(len(data_array))))
        report1 = get_memory_usage()

    start_time_overall = time.time()

    with concurrent.futures.ThreadPoolExecutor() as executor:
        l = [len(data_array)] * len(data_array)
        df_array = list(executor.map(lambda i: read_data(i, data_array[i], l[i], start_time_overall), range(len(data_array))))
        df_array = [x for x in df_array if x is not None]
        report2 = get_memory_usage()

    print(f"Download memory usage\n{report1}\n\n")
    print(f"Pickle memory usage\n{report2}")

    # only when all the threads are complete, then concat happens
    print(len(df_array))
    print(f'Total directory size: {get_directory_size("pickle")}')
    print(f"Download time: {(time_data['url']/3600):.1f} min")
    print(f"Pickle time: {(time_data['pickle']/3600):.1f} min")

    return df_array


def get_directory_size(directory):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(directory):
        for filename in filenames:
            filepath = os.path.join(dirpath, filename)
            total_size += os.path.getsize(filepath)
            total_size_mb = total_size / (1024 * 1024)
    return total_size_mb

def get_memory_usage():

    # Get the memory usage
    memory_usage = psutil.virtual_memory()

    # Print the memory usage
    report = f"""
        Total Memory: {memory_usage.total / (1024 ** 3):.2f} GB
        Available Memory: {memory_usage.available / (1024 ** 3):.2f} GB
        Used Memory: {memory_usage.used / (1024 ** 3):.2f} GB
        Memory Usage Percentage: {memory_usage.percent:.2f}%
    """
    return report

df_array = load_data(dt.datetime(2024,2,1,0,0,0),dt.datetime(2024,2,1,0,10,0))

pickle exists pickle/20240201/000000Z.pkl
pickle exists pickle/20240201/000005Z.pkl
pickle exists pickle/20240201/000015Z.pkl
pickle exists pickle/20240201/000010Z.pkl
pickle exists pickle/20240201/000020Z.pkl
pickle exists pickle/20240201/000025Z.pkl
pickle exists pickle/20240201/000030Z.pkl
pickle exists pickle/20240201/000040Z.pkl
pickle exists pickle/20240201/000035Z.pkl
pickle exists pickle/20240201/000045Z.pkl
pickle exists pickle/20240201/000050Z.pkl
pickle exists pickle/20240201/000105Z.pkl
pickle exists pickle/20240201/000055Z.pkl
pickle exists pickle/20240201/000100Z.pkl
pickle exists pickle/20240201/000110Z.pkl
pickle exists pickle/20240201/000115Z.pkl
pickle exists pickle/20240201/000125Z.pkl
pickle exists pickle/20240201/000135Z.pkl
pickle exists pickle/20240201/000130Z.pkl
pickle exists pickle/20240201/000155Z.pkl
pickle exists pickle/20240201/000145Z.pkl
pickle exists pickle/20240201/000150Z.pkl
pickle exists pickle/20240201/000120Z.pkl
pickle exists pickle/20240201/0002

In [13]:
def run_join_df(df_array):
    start_time = time.time()

    df1 = join_df(df_array[:119])

    time_data["concat"] += time_elapsed(start_time)
    # df1.set_index('time')

    return df1

def join_df(df_array):
    
    df_array2=[]

    if (len(df_array) % 2 == 0):
        for i in range(0, len(df_array)-1, 2):
            df_array2.append(pd.concat([df_array[i], df_array[i+1]], ignore_index=True))
    else:
        for i in range(0, len(df_array)-2, 2):
            df_array2.append(pd.concat([df_array[i], df_array[i+1]], ignore_index=True))
        df_array2.append(df_array[len(df_array)-1])
    
    if (len(df_array2)==1):
        return df_array2
    else:
        return join_df(df_array2)
    
df1 = run_join_df(df_array)
df1

AttributeError: 'list' object has no attribute 'set_index'

In [19]:
print(f"List all type with counts {df1['type'].value_counts()}")
# List all type with counts type
# adsb_icao         9145 - messages from a Mode S or ADS-B transponder, using a 24-bit ICAO address
# other              725 - IGNORE miscellaneous data received via Basestation / SBS format, quality / source is unknown.
# adsb_icao_nt       524 - IGNORE - messages from an ADS-B equipped “non-transponder” emitter e.g. a ground vehicle, using a 24-bit ICAO address
# mode_s             515 - ModeS data from the planes transponder (no position transmitted)
# adsr_icao          280 - rebroadcast of ADS-B messages originally sent via another data link e.g. UAT, using a 24-bit ICAO address
# tisb_other         256 - traffic information about a non-ADS-B target using a non-ICAO address
# tisb_trackfile     214 - traffic information about a non-ADS-B target using a track/file identifier, typically from primary or Mode A/C radar
# mlat               116 - MLAT, position calculated arrival time differences using multiple receivers, outliers and varying accuracy is expected.
# unknown             49
# tisb_icao           31 - traffic information about a non-ADS-B target identified by a 24-bit ICAO address, e.g. a Mode S target tracked by secondary radar
# adsb_other          17 - messages from an ADS-B transponder using a non-ICAO address, e.g. anonymized address
plt.hist(df1['type'], align='left')
plt.title('type')
plt.xticks(rotation='vertical')
plt.show()

TypeError: list indices must be integers or slices, not str

In [ ]:

df1_filtered = df1[df1[['nac_p','nac_v']].notnull().all(1)]
nacp_high = df1_filtered[df1_filtered['nac_p']<8]
nacp_high
# rr_lat not null
# rr_lon not null
# lastPosition.lat / lastPosition.lon not null

In [ ]:
print(f"List all ADSB versions with counts {df1['version'].value_counts()}")
# version: ADS-B Version Number 0, 1, 2 (3-7 are reserved) (2.2.3.2.7.5)
# List all ADSB versions with counts version
# 2.0    8881
# 0.0    1119
# 1.0     141
plt.hist(df1['version'])
plt.title('version')
plt.show()

In [ ]:
# nac_p: Navigation Accuracy for Position (2.2.5.1.35)
counts = df1['nac_p'].value_counts()
print(f"List all nac_p values with counts {counts}")
plt.hist(df1['nac_p'])
plt.title('nac_p')
plt.show()

In [ ]:
# nac_v: Navigation Accuracy for Velocity (2.2.5.1.19)
counts = df1['nac_v'].value_counts()
print(f"List all nac_v values with counts {counts}")
plt.hist(df1['nac_v'])
plt.title('nac_v')
plt.show()

In [ ]:
# roll: Roll, degrees, negative is left roll
counts = df1['roll'].value_counts()
print(f"List all roll with counts {counts}")
plt.hist(df1['roll'], align='left')
plt.title('roll')
plt.show()

In [ ]:
# category: emitter category to identify particular aircraft or vehicle classes (values A0 – D7) (2.2.3.2.5.2)
counts = df1['category'].dropna()
# print(f"List all category with counts {counts}")
plt.hist(counts)
plt.title('category')
plt.show()
# A0 : No ADS-B emitter category information. Do not use this emitter category. If no emitter category fits your installation, seek guidance from the FAA as appropriate. A1 : Light (< 15500 lbs) – Any airplane with a maximum takeoff weight less than 15,500 pounds. This includes very light aircraft (light sport aircraft) that do not meet the requirements of 14 CFR § 103.1.
# A2 : Small (15500 to 75000 lbs) – Any airplane with a maximum takeoff weight greater than or equal to15,500 pounds but less than 75,000 pounds.
# A3 : Large (75000 to 300000 lbs) – Any airplane with a maximum takeoff weight greater than or equal to 75,000 pounds but less than 300,000 pounds that does not qualify for the high vortex category.
# A4 :  High vortex large (aircraft such as B-757) – Any airplane with a maximum takeoff weight greater than or equal to 75,000 pounds but less than 300,000 pounds that has been determined to generate a high wake vortex. Currently, the Boeing 757 is the only example.
# A5 : Heavy (> 300000 lbs) – Any airplane with a maximum takeoff weight equal to or above 300,000 pounds.
# A6 : High performance (> 5g acceleration and 400 kts) – Any airplane, regardless of weight, which can maneuver in excess of 5 G’s and maintain true airspeed above 400 knots.
# A7 : Rotorcraft – Any rotorcraft regardless of weight.
# B0 : No ADS-B emitter category information
# B1 : Glider / sailplane – Any glider or sailplane regardless of weight.
# B2 : Lighter-than-air – Any lighter than air (airship or balloon) regardless of weight.
# B3 : Parachutist / skydiver
# B4 : Ultralight / hang-glider / paraglider – A vehicle that meets the requirements of 14 CFR § 103.1. Light sport aircraft should not use the ultralight emitter category unless they meet 14 CFR § 103.1.
# B5 : Reserved
# B6 : Unmanned aerial vehicle – Any unmanned aerial vehicle or unmanned aircraft system regardless of weight.
# B7 : Space / trans-atmospheric vehicle
# C0 : No ADS-B emitter category information
# C1 : Surface vehicle – emergency vehicle
# C2 : Surface vehicle – service vehicle
# C3 : Point obstacle (includes tethered balloons)
# C4 : Cluster obstacle
# C5 : Line obstacle
# C6 : Reserved
# C7 : Reserved

In [ ]:
# Change in NIC/NAC/SIL indicates an anomaly, which may be due to any reason

# nic: Navigation Integrity Category (2.2.3.2.7.2.6)
# Table 1: NIC value and corresponding size of containment radius
# NIC Containment Radius
# 0 Unknown
# 1 Rc < 37.04km (20nm)
# 2 Rc < 14.816km (8nm)
# 3 Rc < 7.408km (4nm)
# 4 Rc < 3.704km (2nm)
# 5 Rc < 1852m (1nm)
# 6 Rc < 1111.2m (0.6nm)
# Rc < 926m (0.5nm)
# Rc < 555.6m (0.3nm)
# 7 Rc < 370.4m (0.2nm)
# 8 Rc < 185.2m (0.1nm)
# 9 Rc < 75m
# 10 Rc < 25m
# 11 Rc < 7.5m

# Assuming you have already calculated the counts
counts = df1['nic'].value_counts(dropna=False)
print(f"List all nic values with counts {counts}")
# Create bins for the histogram
bins = np.arange(len(counts) + 1)

# Plot the histogram with NA on the left
plt.hist(df1['nic'], bins=bins, align='left',rwidth=0.5)
plt.yscale('log')
plt.title('nic')
plt.show()

In [ ]:
counts = df1['sil'].value_counts(dropna=False)

print(f"List all sil values with counts {counts}")
# Create bins for the histogram
bins = np.arange(len(counts) + 1)

# Plot the histogram with NA on the left
plt.hist(df1['sil'], bins=bins, align='left',rwidth=0.5)
plt.yscale('log')
plt.title('sil')
plt.show()

In [ ]:
url2 = 'https://samples.adsbexchange.com/readsb-hist/2024/02/01/000005Z.json.gz'
response2 = requests.get(url2)
json_data2 = response2.json()
df2 = pd.json_normalize(json_data2['aircraft'])
url3 = 'https://samples.adsbexchange.com/readsb-hist/2024/02/01/000010Z.json.gz'
response3 = requests.get(url3)
json_data3 = response2.json()
df3 = pd.json_normalize(json_data3['aircraft'])
url4 = 'https://samples.adsbexchange.com/readsb-hist/2024/02/01/000015Z.json.gz'
response4 = requests.get(url4)
json_data4 = response2.json()
df4 = pd.json_normalize(json_data4['aircraft'])

In [ ]:
list(df2.columns.values)

In [ ]:
# df2_new = df2[(df2["nic"] > 0 ) & (df2["nic"] <= 14)][["nic","hex",*df2.columns.values]]
df2 = df2[['hex',
 'lat',
 'lon',
 'nic',
 'type',
 'flight',
 'r',
 't',
 'category',
 'version',
 'nac_p',
 'nac_v',
 'roll',
 ]]
df2